In [1]:
import sys
sys.path.append('../../../code/')
sys.path.append('../../../code/michael')
import os
import json
from datetime import datetime
import time
from pipeline.download_data import url_to_dict

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats

import networkx as nx
from load_data import load_citation_network, case_info
import operator
import scotus_viz_functions_michael as viz

from collections import OrderedDict


%load_ext autoreload
%autoreload 2
%matplotlib inline

data_dir = '../../../data/'
court_name = 'scotus'

# assign/make 'scotus_visualizations/michael' directory to save files in
proj_cwd = os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))
viz_dir = os.path.join(proj_cwd, 'visualization')
if not os.path.exists(viz_dir):
    os.makedirs(viz_dir)
viz_scotus_dir = os.path.join(viz_dir, 'scotus')
if not os.path.exists(viz_scotus_dir):
    os.makedirs(viz_scotus_dir)
viz_scotus_michael_dir = os.path.join(viz_scotus_dir, 'michael')
if not os.path.exists(viz_scotus_michael_dir):
    os.makedirs(viz_scotus_michael_dir)
    
csvs_dir = os.getcwd() + '/csvs'

### Create SCOTUS Network (Directed Graph)

In [2]:
# this will be a little slow the first time you run it

time1 = time.time()
G = load_citation_network(data_dir, court_name)
time2 = time.time()

print 'loading scotus network in networkx took %d seconds' % (time2-time1)
print 'loaded %s network with %d cases and %d edges' % (court_name, len(G.nodes()), len(G.edges()))

loading scotus network in networkx took 39 seconds
loaded scotus network with 33248 cases and 250465 edges


### U = Undirected SCOTUS

In [3]:
U = G.to_undirected()

### Acquire Dictionaries for Centrality Measures of SCOTUS

In [4]:
time1 = time.time()
close_cent_dict = nx.closeness_centrality(U) ## according to intuition, graph results, and numerous outside resources
time2  = time.time()
print 'closeness centrality took %d seconds to compute on %d nodes and %d edges' % (time2-time1, U.number_of_nodes(), U.number_of_edges())
eigen_cent_dict = nx.eigenvector_centrality(U) ## according to networkx
time3 = time.time()
print 'eigenvector centrality took %d seconds to compute on %d nodes and %d edges' % (time3-time2, U.number_of_nodes(), U.number_of_edges())
between_cent_dict = nx.betweenness_centrality(U) ## both directed and undirected and ok (both have normalization for networkx)
time4 = time.time()
print 'betweenness centrality took %d seconds to compute on %d nodes and %d edges' % (time4-time3, U.number_of_nodes(), U.number_of_edges())
page_rank_dict = nx.pagerank(G) ## according to networkx
time5 = time.time()
print 'page rank took %d seconds to compute on %d nodes and %d edges' % (time5-time4, G.number_of_nodes(), G.number_of_edges())
hubs_auths_dict = nx.hits(G) ## according to networkx
time6 = time.time()
print 'hubs and authorities took %d seconds to compute on %d nodes and %d edges' % (time6-time5, G.number_of_nodes(), G.number_of_edges())


closeness centrality took 5385 seconds to compute on 33248 nodes and 249878 edges
eigenvector centrality took 21 seconds to compute on 33248 nodes and 249878 edges
betweenness centrality took 22205 seconds to compute on 33248 nodes and 249878 edges
page rank took 4 seconds to compute on 33248 nodes and 250465 edges
hubs and authorities took 33 seconds to compute on 33248 nodes and 250465 edges


### Compare Top 10 Cases for Centrality Measures of SCOTUS

In [5]:
ind = sorted(G.in_degree().iteritems(), key=operator.itemgetter(1), reverse=True)[:10]
ind_cases = [x[0] for x in ind]

outd = sorted(G.out_degree().iteritems(), key=operator.itemgetter(1), reverse=True)[:10]
outd_cases = [x[0] for x in outd]

ccd = sorted(close_cent_dict.iteritems(), key=operator.itemgetter(1), reverse=True)[:10]
ccd_cases = [x[0] for x in ccd]

ecd = sorted(eigen_cent_dict.iteritems(), key=operator.itemgetter(1), reverse=True)[:10]
ecd_cases = [x[0] for x in ecd]

bcd = sorted(between_cent_dict.iteritems(), key=operator.itemgetter(1), reverse=True)[:10]
bcd_cases = [x[0] for x in bcd]

prd = sorted(page_rank_dict.iteritems(), key=operator.itemgetter(1), reverse=True)[:10]
prd_cases = [x[0] for x in prd]

hd = sorted(hubs_auths_dict[0].iteritems(), key=operator.itemgetter(1), reverse=True)[:10]
hd_cases = [x[0] for x in hd]

ad = sorted(hubs_auths_dict[1].iteritems(), key=operator.itemgetter(1), reverse=True)[:10]
ad_cases = [x[0] for x in ad]

data = {'In-Degree': ind_cases,
        'Out-Degree': outd_cases,
        'Closeness Centrality': ccd_cases,
        'Eigenvector Centrality': ecd_cases,
        'Betweenness Centrality': bcd_cases,
        'Page Rank': prd_cases,
        'Hubs': hd_cases,
        'Authorities': ad_cases
       }
df = pd.DataFrame(data, columns = ['In-Degree', 'Out-Degree', 'Closeness Centrality', 'Eigenvector Centrality',
                                   'Betweenness Centrality', 'Page Rank', 'Hubs', 'Authorities'],
                  index=['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4', 'Rank 5', 'Rank 6', 'Rank 7', 'Rank 8', 'Rank 9', 'Rank 10']
                 )

df.to_csv(csvs_dir + '/scotus_centrality_comparison.csv')
df

,In-Degree,Out-Degree,Closeness Centrality,Eigenvector Centrality,Betweenness Centrality,Page Rank,Hubs,Authorities
Rank 1,96405,105210,102605,106514,96405,85131,108611,103355
Rank 2,109532,104616,103012,103355,103012,91573,107082,106514
Rank 3,107252,106366,101864,109380,101864,85534,109380,103243
Rank 4,91573,102224,106366,106761,102605,88661,106267,106761
Rank 5,106545,108329,96405,103243,106366,85160,108839,103347
Rank 6,111221,101864,101894,107082,101894,89675,108798,105746
Rank 7,102605,106267,85330,103347,106447,98094,109505,105751
Rank 8,103012,106548,106170,105751,104616,88804,109836,102991
Rank 9,103355,97966,106447,105746,106545,85330,149702,103870
Rank 10,106761,108221,103950,106142,106548,87010,108329,101097


### Remove Case 96405 from G (To Account for Top 10 In/Out-DegreesList)

In [6]:
G.remove_node(96405)

### Remove Case 96405 from Dictionaries of Centrality Measures

In [12]:
del close_cent_dict[96405]
del eigen_cent_dict[96405]
del between_cent_dict[96405]
del page_rank_dict[96405]
del hubs_auths_dict[0][96405]
del hubs_auths_dict[1][96405]

### Compare Top 10 Cases for Centrality Measures of SCOTUS without Case 96405

In [13]:
ind = sorted(G.in_degree().iteritems(), key=operator.itemgetter(1), reverse=True)[:10]
ind_cases = [x[0] for x in ind]

outd = sorted(G.out_degree().iteritems(), key=operator.itemgetter(1), reverse=True)[:10]
outd_cases = [x[0] for x in outd]

ccd = sorted(close_cent_dict.iteritems(), key=operator.itemgetter(1), reverse=True)[:10]
ccd_cases = [x[0] for x in ccd]

ecd = sorted(eigen_cent_dict.iteritems(), key=operator.itemgetter(1), reverse=True)[:10]
ecd_cases = [x[0] for x in ecd]

bcd = sorted(between_cent_dict.iteritems(), key=operator.itemgetter(1), reverse=True)[:10]
bcd_cases = [x[0] for x in bcd]

prd = sorted(page_rank_dict.iteritems(), key=operator.itemgetter(1), reverse=True)[:10]
prd_cases = [x[0] for x in prd]

hd = sorted(hubs_auths_dict[0].iteritems(), key=operator.itemgetter(1), reverse=True)[:10]
hd_cases = [x[0] for x in hd]

ad = sorted(hubs_auths_dict[1].iteritems(), key=operator.itemgetter(1), reverse=True)[:10]
ad_cases = [x[0] for x in ad]

data = {'In-Degree': ind_cases,
        'Out-Degree': outd_cases,
        'Closeness Centrality': ccd_cases,
        'Eigenvector Centrality': ecd_cases,
        'Betweenness Centrality': bcd_cases,
        'Page Rank': prd_cases,
        'Hubs': hd_cases,
        'Authorities': ad_cases
       }
df = pd.DataFrame(data, columns = ['In-Degree', 'Out-Degree', 'Closeness Centrality', 'Eigenvector Centrality',
                                   'Betweenness Centrality', 'Page Rank', 'Hubs', 'Authorities'],
                  index=['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4', 'Rank 5', 'Rank 6', 'Rank 7', 'Rank 8', 'Rank 9', 'Rank 10']
                 )

df.to_csv(csvs_dir + '/scotus_centrality_comparison_96405_del.csv')
df

,In-Degree,Out-Degree,Closeness Centrality,Eigenvector Centrality,Betweenness Centrality,Page Rank,Hubs,Authorities
Rank 1,109532,105210,102605,106514,103012,85131,108611,103355
Rank 2,107252,104616,103012,103355,101864,91573,107082,106514
Rank 3,91573,106366,101864,109380,102605,85534,109380,103243
Rank 4,106545,102224,106366,106761,106366,88661,106267,106761
Rank 5,111221,108329,101894,103243,101894,85160,108839,103347
Rank 6,102605,101864,85330,107082,106447,89675,108798,105746
Rank 7,103012,106267,106170,103347,104616,98094,109505,105751
Rank 8,103355,106548,106447,105751,106545,88804,109836,102991
Rank 9,106761,97966,103950,105746,106548,85330,149702,103870
Rank 10,85131,108221,96819,106142,85330,87010,108329,101097


### Function to get case_name given case_id

In [41]:
def case_name(case_id):
    """
    Given the case id returns a link to the opinion file on court listener
    """
    url = 'https://www.courtlistener.com/api/rest/v3/clusters/%s/?format=json'\
          % case_id

    case = url_to_dict(url)
    courtlistener_url = 'https://www.courtlistener.com'
    opinion_url = courtlistener_url + case['absolute_url']

    return case['case_name']

### Function to get url given case_id

In [46]:
def case_website(case_id):
    """
    Given the case id returns a link to the opinion file on court listener
    """
    url = 'https://www.courtlistener.com/api/rest/v3/clusters/%s/?format=json'\
          % case_id

    case = url_to_dict(url)
    courtlistener_url = 'https://www.courtlistener.com'
    opinion_url = courtlistener_url + case['absolute_url']

    return opinion_url

### Importance of Cases with Top 10 In-Degree for SCOTUS (Directed) without Case 96405

In [56]:
for each_case in ind_cases:
    print case_name(each_case) , case_website(each_case)

Gregg v. Georgia https://www.courtlistener.com/opinion/109532/gregg-v-georgia/
Miranda v. Arizona https://www.courtlistener.com/opinion/107252/miranda-v-arizona/
Boyd v. United States https://www.courtlistener.com/opinion/91573/boyd-v-united-states/
Gideon v. Wainwright https://www.courtlistener.com/opinion/106545/gideon-v-wainwright/
Chevron USA Inc. v. Natural Resources Defense Council, Inc. https://www.courtlistener.com/opinion/111221/chevron-usa-inc-v-natural-resources-defense-council-inc/
Ashwander v. TVA https://www.courtlistener.com/opinion/102605/ashwander-v-tva/
Erie R. Co. v. Tompkins https://www.courtlistener.com/opinion/103012/erie-r-co-v-tompkins/
Cantwell v. Connecticut https://www.courtlistener.com/opinion/103355/cantwell-v-connecticut/
New York Times Co. v. Sullivan https://www.courtlistener.com/opinion/106761/new-york-times-co-v-sullivan/
The Samuel https://www.courtlistener.com/opinion/85131/the-samuel/


In [63]:
# (oxford importance , legal information institute importance) :
importance = [(1,1), (1,1), (1,0), (1,1), (0,1), (0,1), (1,1), (1,1), (1,1), (0,0)]

oxford = [x[0] for x in importance]
lii = [x[1] for x in importance]

data = {'Cases by In-Degree': ind_cases,
        'Oxford Rating': oxford,
        'LII Rating': lii
       }
df = pd.DataFrame(data, columns = ['Cases by In-Degree', 'Oxford Rating', 'LII Rating'],
                  index=['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4', 'Rank 5', 'Rank 6', 'Rank 7', 'Rank 8', 'Rank 9', 'Rank 10']
                 )

df

,Cases by In-Degree,Oxford Rating,LII Rating
Rank 1,109532,1,1
Rank 2,107252,1,1
Rank 3,91573,1,0
Rank 4,106545,1,1
Rank 5,111221,0,1
Rank 6,102605,0,1
Rank 7,103012,1,1
Rank 8,103355,1,1
Rank 9,106761,1,1
Rank 10,85131,0,0


### Importance of Cases with Top 10 Out-Degree for SCOTUS (Directed) without Case 96405

In [62]:
for each_case in outd_cases:
    print case_name(each_case) , case_website(each_case)

Miller Brothers Co. v. Maryland https://www.courtlistener.com/opinion/105210/miller-brothers-co-v-maryland/
Commissioner v. Estate of Church https://www.courtlistener.com/opinion/104616/commissioner-v-estate-of-church/
Baker v. Carr https://www.courtlistener.com/opinion/106366/baker-v-carr/
Nebbia v. New York https://www.courtlistener.com/opinion/102224/nebbia-v-new-york/
McGautha v. California https://www.courtlistener.com/opinion/108329/mcgautha-v-california/
Crowell v. Benson https://www.courtlistener.com/opinion/101864/crowell-v-benson/
Communist Party of United States v. Subversive Activities Control Bd. https://www.courtlistener.com/opinion/106267/communist-party-of-united-states-v-subversive-activities-control-bd/
Fay v. Noia https://www.courtlistener.com/opinion/106548/fay-v-noia/
The Minnesota Rate Cases https://www.courtlistener.com/opinion/97966/the-minnesota-rate-cases/
Oregon v. Mitchell https://www.courtlistener.com/opinion/108221/oregon-v-mitchell/


In [64]:
# (oxford importance , legal information institute importance) :
importance = [(0,0), (0,0), (1,1), (1,1), (0,0), (0,0), (0,0), (1,0), (0,0), (1,1)]

oxford = [x[0] for x in importance]
lii = [x[1] for x in importance]

data = {'Cases by Out-Degree': outd_cases,
        'Oxford Rating': oxford,
        'LII Rating': lii
       }
df = pd.DataFrame(data, columns = ['Cases by Out-Degree', 'Oxford Rating', 'LII Rating'],
                  index=['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4', 'Rank 5', 'Rank 6', 'Rank 7', 'Rank 8', 'Rank 9', 'Rank 10']
                 )

df

,Cases by Out-Degree,Oxford Rating,LII Rating
Rank 1,105210,0,0
Rank 2,104616,0,0
Rank 3,106366,1,1
Rank 4,102224,1,1
Rank 5,108329,0,0
Rank 6,101864,0,0
Rank 7,106267,0,0
Rank 8,106548,1,0
Rank 9,97966,0,0
Rank 10,108221,1,1


### Importance of Cases with Top 10 Closeness Centrality for SCOTUS (Undirected) without Case 96405

In [61]:
for each_case in ccd_cases:
    print case_name(each_case) , case_website(each_case)

Ashwander v. TVA https://www.courtlistener.com/opinion/102605/ashwander-v-tva/
Erie R. Co. v. Tompkins https://www.courtlistener.com/opinion/103012/erie-r-co-v-tompkins/
Crowell v. Benson https://www.courtlistener.com/opinion/101864/crowell-v-benson/
Baker v. Carr https://www.courtlistener.com/opinion/106366/baker-v-carr/
Burnet v. Coronado Oil & Gas Co. https://www.courtlistener.com/opinion/101894/burnet-v-coronado-oil-gas-co/
Cohens v. Virginia https://www.courtlistener.com/opinion/85330/cohens-v-virginia/
Monroe v. Pape https://www.courtlistener.com/opinion/106170/monroe-v-pape/
Glidden Co. v. Zdanok https://www.courtlistener.com/opinion/106447/glidden-co-v-zdanok/
Yakus v. United States https://www.courtlistener.com/opinion/103950/yakus-v-united-states/
Ex Parte Young https://www.courtlistener.com/opinion/96819/ex-parte-young/


In [65]:
# (oxford importance , legal information institute importance) :
importance = [(0,1), (1,1), (0,0), (1,1), (0,0), (1,1), (0,1), (0,1), (1,0), (1,0)]

oxford = [x[0] for x in importance]
lii = [x[1] for x in importance]

data = {'Cases by Closeness Centrality': ccd_cases,
        'Oxford Rating': oxford,
        'LII Rating': lii
       }
df = pd.DataFrame(data, columns = ['Cases by Closeness Centrality', 'Oxford Rating', 'LII Rating'],
                  index=['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4', 'Rank 5', 'Rank 6', 'Rank 7', 'Rank 8', 'Rank 9', 'Rank 10']
                 )

df

,Cases by Closeness Centrality,Oxford Rating,LII Rating
Rank 1,102605,0,1
Rank 2,103012,1,1
Rank 3,101864,0,0
Rank 4,106366,1,1
Rank 5,101894,0,0
Rank 6,85330,1,1
Rank 7,106170,0,1
Rank 8,106447,0,1
Rank 9,103950,1,0
Rank 10,96819,1,0


### Importance of Cases with Top 10 Eigenvector Centrality for SCOTUS (Undirected) without Case 96405

In [50]:
for each_case in ecd_cases:
    print case_name(each_case) , case_website(each_case)

NAACP v. Button https://www.courtlistener.com/opinion/106514/naacp-v-button/
Cantwell v. Connecticut https://www.courtlistener.com/opinion/103355/cantwell-v-connecticut/
Buckley v. Valeo https://www.courtlistener.com/opinion/109380/buckley-v-valeo/
New York Times Co. v. Sullivan https://www.courtlistener.com/opinion/106761/new-york-times-co-v-sullivan/
Schneider v. State (Town of Irvington) https://www.courtlistener.com/opinion/103243/schneider-v-state-town-of-irvington/
Griswold v. Connecticut https://www.courtlistener.com/opinion/107082/griswold-v-connecticut/
Thornhill v. Alabama https://www.courtlistener.com/opinion/103347/thornhill-v-alabama/
Speiser v. Randall https://www.courtlistener.com/opinion/105751/speiser-v-randall/
NAACP v. Alabama Ex Rel. Patterson https://www.courtlistener.com/opinion/105746/naacp-v-alabama-ex-rel-patterson/
Shelton v. Tucker https://www.courtlistener.com/opinion/106142/shelton-v-tucker/


In [66]:
# (oxford importance , legal information institute importance) :
importance = [(1,1), (1,1), (1,1), (1,1), (0,0), (1,1), (1,1), (0,0), (1,1), (0,1)]

oxford = [x[0] for x in importance]
lii = [x[1] for x in importance]

data = {'Cases by Eigenvector Centrality': ecd_cases,
        'Oxford Rating': oxford,
        'LII Rating': lii
       }
df = pd.DataFrame(data, columns = ['Cases by Eigenvector Centrality', 'Oxford Rating', 'LII Rating'],
                  index=['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4', 'Rank 5', 'Rank 6', 'Rank 7', 'Rank 8', 'Rank 9', 'Rank 10']
                 )

df

,Cases by Eigenvector Centrality,Oxford Rating,LII Rating
Rank 1,106514,1,1
Rank 2,103355,1,1
Rank 3,109380,1,1
Rank 4,106761,1,1
Rank 5,103243,0,0
Rank 6,107082,1,1
Rank 7,103347,1,1
Rank 8,105751,0,0
Rank 9,105746,1,1
Rank 10,106142,0,1


### Importance of Cases with Top 10 Betweenness Centrality for SCOTUS (Undirected) without Case 96405

In [51]:
for each_case in bcd_cases:
    print case_name(each_case) , case_website(each_case)

Erie R. Co. v. Tompkins https://www.courtlistener.com/opinion/103012/erie-r-co-v-tompkins/
Crowell v. Benson https://www.courtlistener.com/opinion/101864/crowell-v-benson/
Ashwander v. TVA https://www.courtlistener.com/opinion/102605/ashwander-v-tva/
Baker v. Carr https://www.courtlistener.com/opinion/106366/baker-v-carr/
Burnet v. Coronado Oil & Gas Co. https://www.courtlistener.com/opinion/101894/burnet-v-coronado-oil-gas-co/
Glidden Co. v. Zdanok https://www.courtlistener.com/opinion/106447/glidden-co-v-zdanok/
Commissioner v. Estate of Church https://www.courtlistener.com/opinion/104616/commissioner-v-estate-of-church/
Gideon v. Wainwright https://www.courtlistener.com/opinion/106545/gideon-v-wainwright/
Fay v. Noia https://www.courtlistener.com/opinion/106548/fay-v-noia/
Cohens v. Virginia https://www.courtlistener.com/opinion/85330/cohens-v-virginia/


In [67]:
# (oxford importance , legal information institute importance) :
importance = [(1,1), (0,0), (0,1), (1,1), (0,0), (0,1), (0,0), (1,1), (1,0), (1,1)]


oxford = [x[0] for x in importance]
lii = [x[1] for x in importance]

data = {'Cases by Betweenness Centrality': bcd_cases,
        'Oxford Rating': oxford,
        'LII Rating': lii
       }
df = pd.DataFrame(data, columns = ['Cases by Betweenness Centrality', 'Oxford Rating', 'LII Rating'],
                  index=['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4', 'Rank 5', 'Rank 6', 'Rank 7', 'Rank 8', 'Rank 9', 'Rank 10']
                 )

df

,Cases by Betweenness Centrality,Oxford Rating,LII Rating
Rank 1,103012,1,1
Rank 2,101864,0,0
Rank 3,102605,0,1
Rank 4,106366,1,1
Rank 5,101894,0,0
Rank 6,106447,0,1
Rank 7,104616,0,0
Rank 8,106545,1,1
Rank 9,106548,1,0
Rank 10,85330,1,1


### Importance of Cases with Top 10 Page Rank for SCOTUS (Directed) without Case 96405

In [52]:
for each_case in prd_cases:
    print case_name(each_case) , case_website(each_case)

The Samuel https://www.courtlistener.com/opinion/85131/the-samuel/
Boyd v. United States https://www.courtlistener.com/opinion/91573/boyd-v-united-states/
Brown v. Maryland https://www.courtlistener.com/opinion/85534/brown-v-maryland/
Slaughter-House Cases https://www.courtlistener.com/opinion/88661/slaughter-house-cases/
Martin v. Hunter's Lessee https://www.courtlistener.com/opinion/85160/martin-v-hunters-lessee/
Davidson v. New Orleans https://www.courtlistener.com/opinion/89675/davidson-v-new-orleans/
Weeks v. United States https://www.courtlistener.com/opinion/98094/weeks-v-united-states/
Ex Parte Lange https://www.courtlistener.com/opinion/88804/ex-parte-lange/
Cohens v. Virginia https://www.courtlistener.com/opinion/85330/cohens-v-virginia/
Murray's Lessee v. Hoboken Land & Improvement Co. https://www.courtlistener.com/opinion/87010/murrays-lessee-v-hoboken-land-improvement-co/


In [70]:
# (oxford importance , legal information institute importance) :
importance = [(0,0), (1,0), (1,0), (0,0), (1,1), (0,0), (1,0), (0,0), (1,1), (1,0)]


oxford = [x[0] for x in importance]
lii = [x[1] for x in importance]

data = {'Cases by Page Rank': prd_cases,
        'Oxford Rating': oxford,
        'LII Rating': lii
       }
df = pd.DataFrame(data, columns = ['Cases by Page Rank', 'Oxford Rating', 'LII Rating'],
                  index=['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4', 'Rank 5', 'Rank 6', 'Rank 7', 'Rank 8', 'Rank 9', 'Rank 10']
                 )

df

,Cases by Page Rank,Oxford Rating,LII Rating
Rank 1,85131,0,0
Rank 2,91573,1,0
Rank 3,85534,1,0
Rank 4,88661,0,0
Rank 5,85160,1,1
Rank 6,89675,0,0
Rank 7,98094,1,0
Rank 8,88804,0,0
Rank 9,85330,1,1
Rank 10,87010,1,0


### Importance of Cases with Top 10 Hubs for SCOTUS (Directed) without Case 96405

In [53]:
for each_case in hd_cases:
    print case_name(each_case) , case_website(each_case)

Branzburg v. Hayes https://www.courtlistener.com/opinion/108611/branzburg-v-hayes/
Griswold v. Connecticut https://www.courtlistener.com/opinion/107082/griswold-v-connecticut/
Buckley v. Valeo https://www.courtlistener.com/opinion/109380/buckley-v-valeo/
Communist Party of United States v. Subversive Activities Control Bd. https://www.courtlistener.com/opinion/106267/communist-party-of-united-states-v-subversive-activities-control-bd/
Paris Adult Theatre I v. Slaton https://www.courtlistener.com/opinion/108839/paris-adult-theatre-i-v-slaton/
Columbia Broadcasting System, Inc. v. Democratic National Committee https://www.courtlistener.com/opinion/108798/columbia-broadcasting-system-inc-v-democratic-national-committee/
Young v. American Mini Theatres, Inc. https://www.courtlistener.com/opinion/109505/young-v-american-mini-theatres-inc/
First Nat. Bank of Boston v. Bellotti https://www.courtlistener.com/opinion/109836/first-nat-bank-of-boston-v-bellotti/
McDonald v. Chicago https://www.co

In [71]:
# (oxford importance , legal information institute importance) :
importance = [(1,1), (1,1), (1,1), (0,0), (1,1), (0,0), (1,1), (1,0), (1,1), (0,0)]

'''
McDonald v. Chicago I'm not sure I noted as 1,1 because of google results ; jurisdiction.csv ends at 2002
'''

oxford = [x[0] for x in importance]
lii = [x[1] for x in importance]

data = {'Cases by Hubs': hd_cases,
        'Oxford Rating': oxford,
        'LII Rating': lii
       }
df = pd.DataFrame(data, columns = ['Cases by Hubs', 'Oxford Rating', 'LII Rating'],
                  index=['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4', 'Rank 5', 'Rank 6', 'Rank 7', 'Rank 8', 'Rank 9', 'Rank 10']
                 )

df

,Cases by Hubs,Oxford Rating,LII Rating
Rank 1,108611,1,1
Rank 2,107082,1,1
Rank 3,109380,1,1
Rank 4,106267,0,0
Rank 5,108839,1,1
Rank 6,108798,0,0
Rank 7,109505,1,1
Rank 8,109836,1,0
Rank 9,149702,1,1
Rank 10,108329,0,0


### Importance of Cases with Top 10 Authorities for SCOTUS (Directed) without Case 96405

In [54]:
for each_case in ad_cases:
    print case_name(each_case) , case_website(each_case)

Cantwell v. Connecticut https://www.courtlistener.com/opinion/103355/cantwell-v-connecticut/
NAACP v. Button https://www.courtlistener.com/opinion/106514/naacp-v-button/
Schneider v. State (Town of Irvington) https://www.courtlistener.com/opinion/103243/schneider-v-state-town-of-irvington/
New York Times Co. v. Sullivan https://www.courtlistener.com/opinion/106761/new-york-times-co-v-sullivan/
Thornhill v. Alabama https://www.courtlistener.com/opinion/103347/thornhill-v-alabama/
NAACP v. Alabama Ex Rel. Patterson https://www.courtlistener.com/opinion/105746/naacp-v-alabama-ex-rel-patterson/
Speiser v. Randall https://www.courtlistener.com/opinion/105751/speiser-v-randall/
Lovell v. City of Griffin https://www.courtlistener.com/opinion/102991/lovell-v-city-of-griffin/
West Virginia Bd. of Ed. v. Barnette https://www.courtlistener.com/opinion/103870/west-virginia-bd-of-ed-v-barnette/
Whitney v. California https://www.courtlistener.com/opinion/101097/whitney-v-california/


In [73]:
# (oxford importance , legal information institute importance) :
importance = [(1,1), (1,1), (0,0), (1,1), (1,1), (1,1), (0,0), (1,0), (1,1), (1,1)]

oxford = [x[0] for x in importance]
lii = [x[1] for x in importance]

data = {'Cases by Authorities': ad_cases,
        'Oxford Rating': oxford,
        'LII Rating': lii
       }
df = pd.DataFrame(data, columns = ['Cases by Authorities', 'Oxford Rating', 'LII Rating'],
                  index=['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4', 'Rank 5', 'Rank 6', 'Rank 7', 'Rank 8', 'Rank 9', 'Rank 10']
                 )

df

,Cases by Authorities,Oxford Rating,LII Rating
Rank 1,103355,1,1
Rank 2,106514,1,1
Rank 3,103243,0,0
Rank 4,106761,1,1
Rank 5,103347,1,1
Rank 6,105746,1,1
Rank 7,105751,0,0
Rank 8,102991,1,0
Rank 9,103870,1,1
Rank 10,101097,1,1
